# 프로젝트 : 얼굴을 인식하여 캐릭터 씌우기
## Face Detection vs Face Recognition
패키지 설치
> pip install mediapipe

In [25]:
import cv2
import mediapipe as mp

def overlay(image, x, y, w, h, overlay_image):  # 대상 이미지(3채널), x, y좌표, width, height, 덮어씌울 이미지(4채널)
    alpha = overlay_image[:, :, 3]  # 4채널 BGRA 중에서 A(alpha, 투명도) 부분만 가져오기 
    mask_image = alpha / 255  # 정규화(0~1) 1:불투명, 0:투명
    
    for c in range(0,3):  # channel BGR
        image[y-h:y+h, x-w:x+w, c] = (overlay_image[:, :, c] * mask_image) + (image[y-h:y+h, x-w:x+w, c] * (1 - mask_image))

# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection  # 얼굴 검출을 위한 face_dectection 모듈을 사용
mp_drawing = mp.solutions.drawing_utils  # 얼굴의 특징을 그리기 위한 drawing_utils 모듈을 사용

# 동영상 파일 열기
cap = cv2.VideoCapture('face_video.mp4')

# 이미지 불러오기
image_right_eye = cv2.imread('right_eye.png', cv2.IMREAD_UNCHANGED)  # (100, 100, 4)
image_left_eye = cv2.imread('left_eye.png', cv2.IMREAD_UNCHANGED)  # (100, 100, 4)
image_nose = cv2.imread('nose.png', cv2.IMREAD_UNCHANGED)  # (100, 300, 4)

with mp_face_detection.FaceDetection(
    model_selection = 0,  # 0:카메라로부터 2미터 이하의 근거리, 1:5미터 이내의 거리
    min_detection_confidence=0.7  # 얼굴로 인식하는 최소 신뢰도 정의(threshold 개념)
) as face_detection:

    while cap.isOpened():  # 동영상 파일이 올바르게 열렸는가?
        success, image = cap.read()  # success : 성공여부, frame : 받아온 이미지 (프레임)
        if not success:
            break
        
        # To improve performance, optionally mark the image as not writeable to pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)
        
        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.detections:
            # 6개 특징 : 눈(우/좌), 코 끝부분, 입 중심, 귀(우/좌)
            for detection in results.detections:
                #mp_drawing.draw_detection(image, detection)
                #print(detection)
                
                # 특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0]  # 오른쪽 눈 x,y좌표
                left_eye = keypoints[1]   # 왼쪽 눈 x,y좌표
                nose_tip = keypoints[2]   # 코 끝부분
                
                h, w, _ = image.shape  # height, width, channel : 이미지로부터 세로, 가로 크기 가져옴.
                right_eye = (int(right_eye.x * w), int(right_eye.y * h))  # 이미지 내에서 실제 좌표 (x, y)
                left_eye = (int(left_eye.x * w), int(left_eye.y * h))
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))
                
                # 양 눈에 동그라미 그리기
                #cv2.circle(image, center=right_eye, radius=50, color=(255,0,0), thickness=10, lineType=cv2.LINE_AA)  # 오른쪽 눈 파란색
                #cv2.circle(image, center=left_eye, radius=50, color=(0,255,0), thickness=10, lineType=cv2.LINE_AA)  # 왼쪽 눈 초록색
                #cv2.circle(image, center=nose_tip, radius=75, color=(0,255,255), thickness=10, lineType=cv2.LINE_AA)  # 코 노란색
                
                # 각 특징에다가 이미지 그리기
                #image[right_eye[1]-110:right_eye[1]+110, right_eye[0]-110:right_eye[0]+110] = image_right_eye
                #image[left_eye[1]-129:left_eye[1]+129, left_eye[0]-129:left_eye[0]+129] = image_left_eye
                #image[nose_tip[1]-50:nose_tip[1]+50, nose_tip[0]-50:nose_tip[0]+50] = image_nose
                
                # image, x, y, w, h, overlay_image
                overlay(image, *right_eye, 50, 50, image_right_eye)
                overlay(image, *left_eye, 50, 50, image_left_eye)
                overlay(image, *nose_tip, 150, 50, image_nose)
                
        # Flip the image horizontally for a self-view display.
        cv2.imshow("MediaPipe Face Detection", cv2.resize(image, None, fx=0.5, fy=0.5))
        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [26]:
image_right_eye = cv2.imread('right_eye.png', cv2.IMREAD_UNCHANGED)  # 투명도 포함(4 channel)
image_right_eye.shape

(100, 100, 4)

In [28]:
image_left_eye = cv2.imread('left_eye.png', cv2.IMREAD_UNCHANGED)
image_left_eye.shape

(100, 100, 4)

In [29]:
image_nose = cv2.imread('nose.png', cv2.IMREAD_UNCHANGED)
image_nose.shape

(100, 300, 4)